In [ ]:
pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

In [ ]:
# Load dataset
instruct_tune_dataset = load_dataset("Isha05/Ielts_reading_text")
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(72))
instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(31))

In [ ]:
def create_prompt(sample):
    """
    Add the prompt and proficiency level to the beginning of the dataset text.
    """
    bos_token = "<s>"
    system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
    question = sample.get("questions")
    text = sample.get("Text")
    eos_token = "</s>"
    
    full_prompt = f"{bos_token}\n### Instruction:\n{system_message}\n\n### Input:\n{question}\n\n### Response:\n{text}\n{eos_token}"
    return [full_prompt]

In [ ]:
# Apply the formatting function to the dataset
instruct_tune_dataset = instruct_tune_dataset.map(lambda sample: {"formatted_text": create_prompt(sample)})

In [ ]:
# Check if a GPU is available
if torch.cuda.is_available():
    device_map = "auto"
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
else:
    device_map = {"": "cpu"}
    nf4_config = None

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map=device_map,
    quantization_config=nf4_config,
    use_cache=False
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda' if torch.cuda.is_available() else 'cpu')
    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded_output = tokenizer.batch_decode(generated_ids)
    return decoded_output[0].replace(prompt, "")

In [ ]:
prompt = """### Instruction: \nUse the provided input to create an instruction that could have been used to generate the response with an LLM.\n\n### Proficiency level:\nBeginner\n\n### Title: My dog\n\n### Text:"""
print(generate_response(prompt, model))

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
    output_dir="mistral_generation",
    max_steps=50,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    warmup_steps=5,
    logging_steps=5,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=2e-4,
    bf16=True,
    lr_scheduler_type='constant',
)
torch.cuda.empty_cache()

In [ ]:
trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    formatting_func=create_prompt,
    args=args,
    train_dataset=instruct_tune_dataset["train"],
    eval_dataset=instruct_tune_dataset["test"]
)

trainer.train()

In [2]:
merged_model = trainer.save_model("mistral_generation")

In [3]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [4]:
# prompt = """### Instruction: \nUse the provided input to create an instruction that could have been used to generate the response with an LLM.\n\n### Proficiency level:\nBeginner\n\n### Title: My dog\n\n### Text:"""
generate_response("""### Instruction: \nUse the provided input to create an instruction that could have been used to generate the response with an LLM in jsonl format where title and text are key and their response is value.\n\n### questions: Write an essay on Global warming negative effects on mandkind having CEFR level C2-advance.\n\n### Text:""", model)

'<s> ### Instruction: \nUse the provided input to create an instruction that could have been used to generate the response with an LLM in jsonl format where title and text are key and their response is value.\n\n### questions: Write an essay on Global warming negative effects on mandkind having CEFR level C2-advance.\n\n### Text: "Global warming is one of the most pressing issues of our time. It is the gradual increase in the average temperature on Earth. This increase in temperature is caused by human activities like burning fossil fuels, deforestation and industrial processes that release greenhouse gases. These greenhouse gases trap heat in the atmosphere and cause the temperature to rise. This rise in temperature has several negative effects on humanity and the ecosystem.\n\nOne of the negative effects of global warming is the change in weather patterns. This change can lead to droughts, floods, hurricanes and tornadoes in different parts of the world. These extreme weather events 